In [1]:
from dataloader import *
from  config import *

SyntaxError: invalid syntax (dataloader.py, line 33)

In [2]:
data = DataLoader(config_mixed_coco2017_coco2017)
# print(config_mixed_flickr8k_flickr8k)

NameError: name 'DataLoader' is not defined

In [ ]:
data.train["train_images_mapping_original"]

In [ ]:
data.train["test_images_mapping_original"]

In [ ]:
len(data.train["all_captions"])

In [ ]:
data.train["train_captions_mapping_original"]

In [ ]:
data.train["test_captions_mapping_original"]

In [ ]:
model=ModelImpl(data)

In [ ]:
model.train()

In [ ]:
model.evaluate()

In [ ]:
generate_report(data.config["results_directory"])